In [12]:
ls = sc.textFile('data/coupon150720.csv', 100) # leemos las lineas del fichero y le decimos que haga 100 particiones

In [14]:
ls = sc.textFile('data/coupon150720.csv')

In [2]:
ls.take(5) # Son cinco lineas aleatorias

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [3]:
ls.first()# coge siempre la fila

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [4]:
ls.count()

1232662

In [5]:
ls.cache() # aqui lo que le dices es que se quede con el calculo que vas a hacer a continuación

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [6]:
ls.count()

1232662

In [7]:
ls.count()

1232662

In [8]:
# Hadoop lee y graba en disco mientras que Spark lo hace en memoria. Con el metodo cache siempre lo hace
# en memoria. los trozos de la RDD se llaman particiones. Se lo podemos indicar o dejar que Spark que lo reparticione

In [10]:
# cache utiliza el metodo por defecto que hayas puesto, es decir, en memoria. Las particiones es un balance entre 
# el rendimiento y la memoria del nodo. Tanto core que haya en un cluster hacemos ese numero de particiones. Cada
#que hay una particion hay trafico entre nodos del cluster para seguir utilizandolo en el siguiente paso.Lo ideal
# es tener pocas particiones 

In [16]:
ls.getNumPartitions()

3

In [17]:
# Cuantos pasajeros que han volado en vuelos operados por Iberia

In [41]:
aerolineas = ls.map(lambda x: x.split(",")[5]) # Primeros tenemos que cortar la cadena de texto y nos quedamos con la columna 5

In [39]:
aerolineas.take(5)

[[u'79062005698500',
  u'1',
  u'MAA',
  u'AUH',
  u'9W',
  u'9W',
  u'56.79',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'0526',
  u'150904',
  u'OK',
  u'IAF0'],
 [u'79062005698500',
  u'2',
  u'AUH',
  u'CDG',
  u'9W',
  u'9W',
  u'84.34',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'6120',
  u'150905',
  u'OK',
  u'IAF0'],
 [u'79062005924069',
  u'1',
  u'CJB',
  u'MAA',
  u'9W',
  u'9W',
  u'60.0',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'2768',
  u'150721',
  u'OK',
  u'IAA0'],
 [u'79065668570385',
  u'1',
  u'DEL',
  u'DXB',
  u'9W',
  u'9W',
  u'160.63',
  u'USD',
  u'2',
  u'S',
  u'S',
  u'0546',
  u'150804',
  u'OK',
  u'INA0'],
 [u'79065668737021',
  u'1',
  u'AUH',
  u'IXE',
  u'9W',
  u'9W',
  u'152.46',
  u'USD',
  u'1',
  u'V',
  u'V',
  u'0501',
  u'150803',
  u'OK',
  u'INA0']]

In [42]:
aerolineas.filter(lambda x: x == "IB").count() # 26000 pasajeros han volado con Iberia

26158

In [ ]:
# AHora vamos a ver cuanto dinero han supuesto para Iberia sus pasajeros

In [43]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)

In [44]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [45]:
dinero = ls.map(getAerolineaDinero)# No hace falta que le pongamos la lambda porque la función ya la tenemos definida

In [55]:
dinerosIB= dinero.filter(lambda x: x[0] == "IB")

In [56]:
cantidades = dinerosIB.map(lambda x: x[1]) # nos quedamos solo con la variable dinero de la lista ya filtrada

In [58]:
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [60]:
round(cantidades.sum())

2828044.0

In [61]:
# ver cual es la ruta que tiene el billete más caro

In [62]:
def getRutaDinero(l):
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta, dinero)
                    

In [63]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [65]:
rutas = ls.map(getRutaDinero)

In [69]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5) # nos ordena por el valor que esta en la posicion 1 que es la ruta. False significa 
# ordena descendente. Nos da el top 5 de cupones más caro, no de las rutas 

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [70]:
filtrado = rutas.filter(lambda x: x[1] < 7500) # Como vemos que hay valores repetidos vamos a organizar por rutas
    

In [76]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [77]:
maximos.sortBy(lambda x: x[1], False). take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [78]:
# La ruta por con el cupon medio más alto 

In [79]:
rutas = ls.map(getRutaDinero)

In [84]:
mean = filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs))

In [85]:
mean.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [87]:
#Definimos las columnas del fichero que nos interesan con una función. Agrupamos los datos por Key(valores no numericos)

In [91]:
def getRutaALDineroClase(l):
    elems = l.split((","))
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta, al, clase), dinero)

In [92]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [93]:
import math

In [ ]:
# Ahora creamos una función para calcular los valores limites, que va a ser dos veces la media 

In [119]:
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1

In [100]:
valorLimite([12,3,4,1,2,4])

14.668153608222282

In [131]:
rutas = ls.map(getAerolineaDinero)

In [121]:
def filtradoCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [122]:
grupos = rutas.groupByKey().mapValues(filtradoCuponesExtremos)

In [125]:
def mediaGrupo(vs):
    if(len(vs) == 0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [126]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [132]:
grupos = rutas.groupByKey()

In [135]:
gruposFiltrados = grupos.filter(lambda x: len(x[1])>10)

In [136]:
r = gruposFiltrados.mapValues(filtradoCuponesExtremos).mapValues(mediaGrupo)

In [137]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

In [138]:
#AHora vamos a ver cuantas aerolineas distintas hay en el dataset

In [140]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [141]:
#Para ver si los valores de los precios estan USD


In [142]:
ls.map(lambda x: x.split(",")[7]).distinct().collect() # el collect nos estaría cargando los datos en memoria del RDD

[u'USD']

In [143]:
# Y si queremos sacar estos datos para luego tratarlos en R?

In [150]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("resultados.csv") 

In [147]:
# Nos va a crear un directorio 
# que se llama resultados y dentro de esa carpeta tenemos un fichero para cada una de las particiones